In [34]:
import os
from pinecone import Pinecone

In [35]:
index_name = "movie-db"

In [36]:
api_key = os.environ.get("PINECONE_API_KEY") or "PINECONE_API_KEY"
pc = Pinecone(api_key=api_key)

In [37]:
if pc.has_index(name=index_name):
    pc.delete_index(name=index_name)

In [38]:
from pinecone import ServerlessSpec, Metric, CloudProvider, AwsRegion

pc.create_index(
    name=index_name,
    dimension=2,
    metric=Metric.COSINE,
    spec=ServerlessSpec(cloud=CloudProvider.AWS, region=AwsRegion.US_EAST_1),
)

{
    "name": "movie-db",
    "metric": "cosine",
    "host": "movie-db-04dftrb.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 2,
    "deletion_protection": "disabled",
    "tags": null
}

In [39]:
import pandas as pd

df = pd.DataFrame()
df['ID'] = ["mad-max", "superbad"]
df['Vector'] = [[0.9, 0.1], [0.1, 0.9]]
df

,ID,Vector
0,mad-max,"[0.9, 0.1]"
1,superbad,"[0.1, 0.9]"


In [40]:
index_config = pc.describe_index(name=index_name)
index = pc.Index(host=index_config.host)
index.upsert(vectors=[('mad-max', [0.9, 0.1])], namespace='action')
index.upsert(vectors=[('superbad', [0.1, 0.9])], namespace='comedy')

{'upserted_count': 1}

In [41]:
index.describe_index_stats()

{'dimension': 2,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'action': {'vector_count': 1}, 'comedy': {'vector_count': 1}},
 'total_vector_count': 2,
 'vector_type': 'dense'}

In [43]:
index.query(top_k=2, vector=[0.9, 0.1])

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}